In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import os
import matplotlib.pyplot as plt
from configs import project_config
import matplotlib.pyplot as plt
import seaborn as sns
from configs.RecursiveVPSDE.Markovian_fBiPotDDims_NonSep.recursive_Markovian_PostMeanScore_fBiPot12DimsNS_T256_H05_tl_110data_StbleTgt import get_config


In [ ]:
num_paths = 10240
root_dir ="/Users/marcos/Library/CloudStorage/OneDrive-ImperialCollegeLondon/StatML_CDT/Year2/DiffusionModels/"
config = get_config()

In [ ]:
def get_drift_files(config, root_dir):
    ts_type = f"{config.ndims}DDims"
    include = (f"_{num_paths}NPaths_{config.t0}t0_{config.deltaT:.3e}dT_").replace(".", "")
    root_nadaraya_dir = root_dir + f"ExperimentResults/Nadaraya/12DDimsNS/"
    true_files = []
    local_files = []
    global_files = []
    for file in os.listdir(root_nadaraya_dir):
        print(file)
        if "IIDNadaraya" in file and ts_type in file and include in file:
            if "true" in file:
                true_files.append(root_nadaraya_dir+file)
            elif "global" in file:
                global_files.append(root_nadaraya_dir+file)
            elif "local" in file:
                local_files.append(root_nadaraya_dir+file)
    assert len(true_files)>0, "No true files found"
    assert len(local_files)>0, "No local files found"
    def extract_bw_number(s):
        match = s.split("/")[-1].split("_")[4][:-2]
        return float("0." + match)
    true_files = sorted(true_files, key=extract_bw_number)
    local_files = sorted(local_files, key=extract_bw_number)
    global_files = sorted(global_files, key=extract_bw_number)
    bws = [extract_bw_number(f) for f in true_files]
    return true_files, local_files, global_files, bws

In [ ]:
true_nadaraya_files, local_nadaraya_files, global_nadaraya_files, bws = get_drift_files(config=config, root_dir=root_dir)

In [ ]:
def get_score_drift_files(root_dir, ts_type):
    root_score_dir = root_dir + f"ExperimentResults/TSPM_Markovian/12DDimsNS/"
    driftoostrack_true_files = []
    driftoostrack_local_files = []
    driftoostrack_global_files = []
    for file in os.listdir(root_score_dir):
        if "_MLP_" in file and ts_type in file:
            if "DriftTrack" in file and "true" in file:
                driftoostrack_true_files.append(root_score_dir+file)
            elif "DriftTrack" in file and "global" in file:
                driftoostrack_global_files.append(root_score_dir+file)
            elif "DriftTrack" in file and "local" in file:
                driftoostrack_local_files.append(root_score_dir+file)
    assert len(driftoostrack_true_files)>0, "No oos drift track files found"
    assert len(driftoostrack_local_files)>0, "No oos drift track files found"
    assert(len(driftoostrack_true_files) == len(driftoostrack_true_files))
    def extract_bw_drift_track_number(s):
        match = s.split("Nep_")[0].split("_")[-1]
        return int(match)
    driftoostrack_true_files = sorted(driftoostrack_true_files, key=extract_bw_drift_track_number)
    driftoostrack_local_files = sorted(driftoostrack_local_files, key=extract_bw_drift_track_number)
    driftoostrack_global_files = sorted(driftoostrack_global_files, key=extract_bw_drift_track_number)
    Nepochs_track = [extract_bw_drift_track_number(f) for f in driftoostrack_true_files]
    idx = np.argwhere(np.array(Nepochs_track)==18).flatten()[0]
    driftoostrack_true_files = driftoostrack_true_files[idx]
    driftoostrack_local_files = driftoostrack_local_files[idx]
    driftoostrack_global_files = driftoostrack_global_files[idx]
    return driftoostrack_true_files, driftoostrack_local_files,driftoostrack_global_files, Nepochs_track

In [ ]:
from configs.RecursiveVPSDE.Markovian_fBiPotDDims_NonSep.recursive_Markovian_PostMeanScore_fBiPot12DimsNS_T256_H05_tl_110data_StbleTgt import get_config as get_score_config
score_config = get_score_config()
ts_type = f"ST_{config.feat_thresh:.3f}FTh_fBiPot_{config.ndims}DDimsNS".replace(".", "")
true_score_files, local_score_files, global_score_files, Nepochs_track = get_score_drift_files(ts_type=ts_type, root_dir=root_dir)

In [ ]:
local_score_files

In [ ]:
score_true = np.load(true_score_files)
score_global = np.load(global_score_files)
nad_true = np.load(true_nadaraya_files[0])
nad_global = np.load(global_nadaraya_files[0])
score_true=score_true.reshape(-1,score_true.shape[2], score_true.shape[-1])
score_global=score_global.reshape(-1,score_global.shape[2], score_global.shape[-1])
nad_true=nad_true.reshape(-1,nad_true.shape[2], nad_true.shape[-1])
nad_global=nad_global.reshape(-1,nad_global.shape[2], nad_global.shape[-1])
true_paths = np.load(score_config.data_path)

In [ ]:
for d in range(score_global.shape[-1]):
    print(nad_true[:,:, d].flatten().shape)
    plt.hist(score_true[:,1:, d].flatten(), bins=150, color="blue",density=True,label="Score")
    plt.hist(score_global[:,1:, d].flatten(), bins=150, color="red",alpha=0.3,density=True,label="Nad")
    #plt.hist(true_paths[:,:, d].flatten(), bins=150, color="red",density=True,label="True")
    plt.legend()
    plt.show()
    plt.close()

In [ ]:
for d in range(score_global.shape[-1]):
    print(nad_true[:,:, d].flatten().shape)
    plt.hist(score_global[:,:, d].flatten(), bins=150, color="blue",density=True,label="Score")
    plt.hist(nad_global[:,:, d].flatten(), bins=150, color="red",alpha=0.3,density=True,label="Nad")
    #plt.hist(true_paths[:,:, d].flatten(), bins=150, color="red",density=True,label="True")
    plt.legend()
    plt.show()
    plt.close()

In [ ]:
incs = np.concatenate([true_paths[:, [0],:],np.diff(true_paths,axis=1)],axis=1).reshape(-1, true_paths.shape[-1])
pos = true_paths[:, :-1, :].reshape(-1, true_paths.shape[-1])

In [ ]:
for k in range(1000):
    Xtj1 = pos[np.random.choice(np.arange(pos.shape[0]), size=1),:]
    valid_idxs = np.sum(np.power(pos - Xtj1,2), axis=-1)/np.sqrt(pos.shape[-1]) <=0.1
    if np.sum(valid_idxs)>5000:
        valid_incs = incs[np.argwhere(valid_idxs).flatten(), :]
        valid_incs_noised = np.exp(-0.25*0.5**2*20)*valid_incs + np.sqrt(1.-np.exp(-0.5*0.5**2*20))*np.random.normal(size=valid_incs.shape)
        for d in range(0,pos.shape[-1], 4):
            plt.hist(pos[np.argwhere(valid_idxs).flatten(), d], label=rf"Dim {d} \pi_0",bins=150)
        plt.legend()
        plt.show()
        plt.close()
        continue
        for d in range(0,pos.shape[-1], 4):
            plt.hist(valid_incs[:, d], label=rf"Dim {d} \pi_0",bins=150)
        plt.legend()
        plt.show()
        plt.close()
        for d in range(0,pos.shape[-1], 4):
            plt.hist(valid_incs_noised[:, d], label=rf"Dim {d} Transition",bins=150)
        plt.legend()
        plt.show()
        plt.close()

In [ ]:
scoreModel = ConditionalMarkovianTSPostMeanScoreMatching(*config.model_parameters)
diffusion = VPSDEDiffusion(beta_max=config.beta_max, beta_min=config.beta_min)
print(
    config.tdata_mult * sum(p.numel() for p in scoreModel.parameters() if p.requires_grad) / (config.ts_length - 1))
init_experiment(config=config)
end_epoch = max(config.max_epochs)
try:
    scoreModel.load_state_dict(torch.load(config.scoreNet_trained_path + "_NEpEE" + str(end_epoch)))